In [1]:
import pandas as pd
import re
import emoji
from underthesea import word_tokenize
import unicodedata
import json
from sklearn.model_selection import train_test_split

print(" Libraries installed and ready.")

d:\UIT-K18\NAM3\HK1\CS221-Natural_Language_Processing\Final-Project\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


 Libraries installed and ready.


In [2]:
def load_data(file_path):
    data = []
    with open(file_path, 'r', encoding='utf-8') as f:
        for line in f:
            data.append(json.loads(line))
    return pd.DataFrame(data)

# Load Train and Test
train_df = load_data('../data/train_augmented.jsonl')
test_df = load_data('../data/test.jsonl')

print(f"Original Train Shape: {train_df.shape}")
print(f"Original Test Shape: {test_df.shape}")

Original Train Shape: (2329, 2)
Original Test Shape: (494, 2)


In [3]:
# Dictionary to fix slang
teencode_dict = {
    "k": "không", "ko": "không", "hok": "không", "khong": "không", "kg": "không",
    "dc": "được", "đc": "được", "duoc": "được",
    "vn": "việt nam", "vnam": "việt nam",
    "ngân hàng": "ngân hàng", "nh": "ngân hàng",
    "tk": "tài khoản", "t khoản": "tài khoản",
    "dv": "dịch vụ",
    "add": "admin", "ad": "admin",
    "rep": "trả lời",
    "nt": "nhắn tin",
    "dt": "điện thoại", "sdt": "số điện thoại",
    "mn": "mọi người",
    "m": "mình", "mk": "mình", "mik": "mình",
    "bit": "biết", "bik": "biết",
    "gud": "tốt", "good": "tốt",
    "thanks": "cảm ơn", "tks": "cảm ơn", "thank": "cảm ơn", "iu": "yêu",
    "wá": "quá", "wa": "quá",
    "z": "vậy", "zậy": "vậy"
}

def replace_teencode(text):
    # Split text into words, replace if in dict, join back
    return " ".join([teencode_dict.get(word.lower(), word) for word in text.split()])

print(" Teencode Dictionary Loaded.")

 Teencode Dictionary Loaded.


In [4]:
def preprocess_text(text):
    # 0. Safety Check
    if not isinstance(text, str) or text.strip() == "":
        return ""
    
    # 1. Unicode Normalization
    text = unicodedata.normalize('NFC', text)
    
    # 2. Lowercase
    text = text.lower()
    
    # 3. Basic Cleaning (HTML, URLs)
    text = re.sub(r'http\S+|www\.\S+', '', text)
    text = re.sub(r'<.*?>', '', text)
    
    # 4. Word Elongation Fix (NEW)
    # Look for 3 or more repeated characters (e.g., "ngoooon") and replace with 1
    # We use \1 to refer to the captured character.
    text = re.sub(r'([a-zA-Z])\1{2,}', r'\1', text)
    
    # 5. Remove Duplicate Punctuation
    # "!!!!" -> "!"
    text = re.sub(r'([!?.])\1+', r'\1', text)
    
    # but for bank names, we usually just remove the '@' symbol.
    text = re.sub(r'[@#]', '', text)

    # 6. Demojize
    text = emoji.demojize(text, delimiters=(" :", ": "))
    
    # 7. Teencode Correction
    text = replace_teencode(text)
    
    # 8. Word Segmentation
    text = word_tokenize(text, format="text")
    
    return text.strip()

# Test the Elongation
print(preprocess_text("Dịch vụ tốt quá đi trờiiiiiii ơi !!!"))
# Expected: "dịch_vụ tốt quá đi trời ơi !"

dịch_vụ tốt quá đi trời_ơi !


In [5]:
# Define the mapping manually to ensure specific order
label_map = {
    'negative': 0,
    'neutral': 1,
    'positive': 2
}

# Apply mapping
print("⏳ Encoding Labels...")
train_df['label'] = train_df['sentiment'].map(label_map)
test_df['label'] = test_df['sentiment'].map(label_map)

# Check if any label failed to map (sanity check)
if train_df['label'].isnull().sum() > 0:
    print(" WARNING: Some rows have unknown sentiments!")
    display(train_df[train_df['label'].isnull()])
    # Drop them if necessary
    train_df = train_df.dropna(subset=['label'])
    test_df = test_df.dropna(subset=['label'])
    
# Convert to integer (just to be safe)
train_df['label'] = train_df['label'].astype(int)
test_df['label'] = test_df['label'].astype(int)

print(" Labels Encoded: negative=0, neutral=1, positive=2")
display(train_df[['sentiment', 'label']].head())

⏳ Encoding Labels...
 Labels Encoded: negative=0, neutral=1, positive=2


,sentiment,label
0,negative,0
1,negative,0
2,negative,0
3,negative,0
4,negative,0


In [6]:
# --- 1. Apply Preprocessing ---
print(" Processing Train Data (Original)...")
train_df['text_clean'] = train_df['text'].apply(preprocess_text)

print(" Processing Test Data...")
test_df['text_clean'] = test_df['text'].apply(preprocess_text)

# --- 2. Clean & Deduplicate ---
# Drop duplicates (Smart Deduplication: Keep 1, drop rest)
train_df = train_df.drop_duplicates(subset=['text_clean'], keep='first')
test_df = test_df.drop_duplicates(subset=['text_clean'], keep='first')

# Drop empty rows (Garbage)
train_df = train_df[train_df['text_clean'].str.strip() != '']
test_df = test_df[test_df['text_clean'].str.strip() != '']

print(f"Rows after cleaning: {len(train_df)}")

# --- 3. Create Validation Split (NEW) ---
# We split the Cleaned Train data: 85% Train, 15% Validation.
# stratify=train_df['label'] ensures both sets have the same % of Negative/Positive classes.
print(" Splitting Train into Train & Validation sets...")
train_df, val_df = train_test_split(
    train_df,
    test_size=0.15,
    stratify=train_df['label'], # Uses the 'label' column created in the previous cell
    random_state=42
)

# --- 4. Save to CSV ---
train_df.to_csv("../data/processed_aug/train_processed.csv", index=False)
val_df.to_csv("../data/processed_aug/val_processed.csv", index=False)
test_df.to_csv("../data/processed_aug/test_processed.csv", index=False)

print("\ Final Data Saved: 'train_processed.csv', 'val_processed.csv', 'test_processed.csv'")
print(f"• Final Train Shape: {train_df.shape}")
print(f"• Final Val Shape:   {val_df.shape}")
print(f"• Final Test Shape:  {test_df.shape}")

 Processing Train Data (Original)...


<>:35: SyntaxWarning: invalid escape sequence '\ '
<>:35: SyntaxWarning: invalid escape sequence '\ '
C:\Users\tntdu\AppData\Local\Temp\ipykernel_20200\2745148330.py:35: SyntaxWarning: invalid escape sequence '\ '
  print("\ Final Data Saved: 'train_processed.csv', 'val_processed.csv', 'test_processed.csv'")


 Processing Test Data...
Rows after cleaning: 2167
 Splitting Train into Train & Validation sets...
\ Final Data Saved: 'train_processed.csv', 'val_processed.csv', 'test_processed.csv'
• Final Train Shape: (1841, 4)
• Final Val Shape:   (326, 4)
• Final Test Shape:  (464, 4)


In [7]:
train_df.head()

,text,sentiment,label,text_clean
1321,Ko ăn thua. Làm sao mà là số 1 đc. Khi ngày 15...,negative,0,không ăn_thua . làm_sao mà là số 1 đc . khi ng...
2098,Không có đủ trình độ để đạt được mục tiêu đề ra,neutral,1,không có đủ trình_độ để đạt được mục_tiêu_đề ra
1274,Chúc mừng sự phát triển của BIDV Việt Nam!,positive,2,chúc_mừng sự phát_triển của bidv việt_nam !
2037,"Câu Không có tên, buồn quá! không phù hợp với ...",neutral,1,"câu không có tên , buồn quá ! không phù_hợp vớ..."
1853,trò này không hay và hấp dẫn mà cũng ko công b...,negative,0,trò này không hay và hấp_dẫn mà cũng không côn...


In [8]:
val_df.head()

,text,sentiment,label,text_clean
2203,Tôi muốn biết thêm về cách đăng ký dịch vụ int...,neutral,1,tôi muốn biết thêm về cách đăng_ký dịch_vụ int...
1565,Chiều nay mình có qua điểm giao dịch vietcomba...,negative,0,chiều nay mình có qua điểm giao_dịch vietcomba...
1559,Không chỉ riêng chi nhánh nào. Mình đi các vie...,negative,0,không_chỉ riêng chi_nhánh nào . mình đi các vi...
170,Rất hay ạ :),positive,2,rất hay ạ :)
1702,Tin tưởng ngân hàng Vietcombank mới mở tài kho...,negative,0,tin_tưởng ngân_hàng vietcombank mới mở tài_kho...


In [9]:
test_df.head()

,text,sentiment,label,text_clean
0,Gọi k được mà tốn tiền như gì ấy,negative,0,gọi không được mà tốn tiền như gì ấy
1,"12h đêm thì sao nghẽn hả ad, mình cũng từng là...",negative,0,"12 h đêm thì sao nghẽn hả_ad , mình cũng từng ..."
2,Vietcombank ngân hàng tốt,positive,2,vietcombank ngân_hàng tốt
3,Chơi hoài mà không có trúng gì hết,neutral,1,chơi hoài mà không có trúng gì hết
4,Vietcom ít cây ATM quá :(,negative,0,vietcom ít cây atm quá : (
